In [1]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances

In [16]:
dirs="/data02/tguo/space_batch_effect/mouse_OB/"
sample_name=["BGI","SlideV2","10X"]
sample_name=["BGI","SlideV2","scRNA"]
flags1='_'+sample_name[0]+'_'+sample_name[1]
flags2='_'+sample_name[0]+'_'+sample_name[1]+'_'+sample_name[2]
clusters=pd.read_csv(dirs+'metrics/louvain_clust'+flags1+'.csv',index_col=0,header=0)
# coord=pd.read_csv(dirs+"gtt_input/10X_coord.csv",header=0,index_col=0,sep=',')
embed_all=pd.read_csv(dirs+'gtt_output/AGC_embed'+flags2+"_expression_xent.csv",header=0,index_col=0,sep=',')
if embed_all.shape[1]==16:
    znoise_dim=2
else:
    znoise_dim=4
embed_all=embed_all.iloc[:,znoise_dim:]
meta=pd.read_csv(dirs+"gtt_input/"+str(sample_name[0])+"_meta.csv",header=0,index_col=0,sep=',')
for sample in np.array(sample_name)[1:len(sample_name)]:
    meta=pd.concat((meta,pd.read_csv(dirs+"gtt_input/"+str(sample)+"_meta.csv",header=0,index_col=0,sep=',')))
meta['celltype'][meta['celltype']=='SEZ']='GCL'
meta['celltype'][meta['celltype']=='Meninges']='ONL'
meta['celltype'][meta['celltype']=='OPL']='EPL'
cells=meta.index[meta['celltype']!='Low_Quality']
embed_all=embed_all.loc[cells,:]
meta=meta.loc[cells,]
meta=meta.loc[cells,]
idx1=np.where(meta['batch']!=sample_name[2])[0]
idx2=np.where(meta['batch']==sample_name[2])[0]
embed_train=embed_all.iloc[idx1,:]
embed_test=embed_all.iloc[idx2,:]
clusters=clusters.loc[embed_train.index,:]


In [18]:
KNN=6
KNN=10
KNC=KNeighborsClassifier(n_neighbors=KNN)
KNC.fit(embed_train.values,clusters.values[:,0])
cluster_pred=KNC.predict(embed_test.values)
cluster_pred=pd.DataFrame(cluster_pred,index=embed_test.index,columns=['pred_clusters'])
cluster_pred.to_csv(dirs+'metrics/'+sample_name[2]+'_expression_pred_clusters.csv')

In [ ]:
celltype=pd.read_csv(dirs+"gtt_input/"+sample_name[2]+"_meta.csv",header=0,index_col=0,sep=',')
celltype=celltype.iloc[:,0].values

In [22]:
from sklearn.metrics.cluster import adjusted_rand_score
adjusted_rand_score(celltype,cluster_pred.values[:,0])

0.2449800028803275

In [4]:
def refine(sample_id, pred, dis, shape="hexagon"):
    refined_pred=[]
    pred=pd.DataFrame({"pred": pred}, index=sample_id)
    dis_df=pd.DataFrame(dis, index=sample_id, columns=sample_id)
    if shape=="hexagon":
        num_nbs=6 
    elif shape=="square":
        num_nbs=4
    else:
        print("Shape not recongized, shape='hexagon' for Visium data, 'square' for ST data.")
    for i in range(len(sample_id)):
        index=sample_id[i]
        dis_tmp=dis_df.loc[index, :].sort_values()
        nbs=dis_tmp[0:num_nbs+1]
        nbs_pred=pred.loc[nbs.index, "pred"]
        self_pred=pred.loc[index, "pred"]
        v_c=nbs_pred.value_counts()
        if (v_c.loc[self_pred]<(num_nbs+1)/2) and (np.max(v_c)>(num_nbs+1)/2):
            refined_pred.append(v_c.idxmax())
        else:           
            refined_pred.append(self_pred)
    return refined_pred

In [5]:
pred=cluster_pred
dis=euclidean_distances(coord,coord)
refined_pred=refine(np.array(pred.index), pred.values[:,0], dis, shape="hexagon")
refined_pred=pd.DataFrame(refined_pred,index=embed_test.index,columns=['pred_clusters'])
refined_pred.to_csv(dirs+'metrics/10X_pred_clusters.csv')

In [19]:
celltype=pd.read_csv(dirs+"gtt_input/"+sample_name[2]+"_meta.csv",header=0,index_col=0,sep=',')
celltype=celltype.iloc[:,0].values

In [21]:
from sklearn.metrics.cluster import adjusted_rand_score
adjusted_rand_score(celltype,refined_pred.values[:,0])

ValueError: Found input variables with inconsistent numbers of samples: [21746, 1185]

In [20]:
celltype

array(['Granule_cell', 'Granule_cell', 'Granule_cell', ...,
       'Mitral_and_tufted_cell', 'Mitral_and_tufted_cell',
       'Mitral_and_tufted_cell'], dtype=object)